## Titiler / STAC Demo

This Notebook aims to show the different features provided by titiler application.

First it will get a list of STAC Items available at a set of coordinates and display their boundaries on a map.

Then it will print out the available assets and their information so we can chose which item and asset we want to view.

After we select an item and asset, we will view it on a sample map using the information provided previously.

You can view images based on pre-set search queries (ie query by STAC item metadata parameters such as collection, bounding box, acquisistion time, cloud cover, vendor, sensor, etc).  

Or you can view images based on STAC Item ID and Asset name.

### Python Requirements
```
pip install httpx folium pypgstac psycopg psycopg-pool geojson-pydantic pandas mercantile
```

In [1]:
import json
import httpx
import pandas as pd
from folium import Map, TileLayer, GeoJson, raster_layers, LayerControl

from IPython.display import HTML, display

from geojson_pydantic import Feature, Polygon

#endpoint = "https://gis.roycegeo.com/pgtitiler"
endpoint = "https://titiler.grvty.com/pgtitiler"

print(httpx.get(f"{endpoint}/healthz").json())

{'database_online': True, 'versions': {'titiler': '0.21.1', 'titiler.pgstac': '1.7.0', 'rasterio': '1.4.3', 'gdal': '3.9.3', 'proj': '9.4.1', 'geos': '3.11.1'}}


### Show the bounding boxes of available Items at a set of coordinates on a map

Using a path in the Collections group, were we supply the collection name (ares-demo) and a set of coordinates, titiler will return a list of STAC items at that location in the collection.

We can use the geometry of each returned item to display on a map.

In [3]:
# Get the available Items at coordinates -63.403946,2.355549

bounds = (-63.58651954366129, 2.1766122457878, -63.25108334986687, 2.4148986049920276)
m = Map(
    tiles="OpenStreetMap",
    location=((bounds[1] + bounds[3]) / 2, (bounds[0] + bounds[2]) / 2),
    zoom_start=8,
)

response = httpx.get(
    f"{endpoint}/collections/royce-dev/point/-60.600332,3.968695/assets"
).json()

#  locations 
# -63.403946,2.355549
# -63.04044583,2.630686308
# -64.1718224,3.4175373
# -60.600332,3.968695
# -61.771634,5.955525

for item in response:
    bbox = item['bbox']
    poly = Polygon.from_bounds(bbox[0],bbox[1],bbox[2],bbox[3])
    geojson = Feature(type="Feature", geometry=poly, properties=None).model_dump(exclude_none=True)
    geo_json = GeoJson(
        data=geojson,
        style_function=lambda x: {
            "opacity": 1,
            "dashArray": "1",
            "fillOpacity": 0,
            "weight": 1,
            },
        )
    geo_json.add_to(m)


m

### View information about each STAC Item.

We are displaying the about each STAC item returned.  We can use this information to create a query to sent to the Titiler api to display the image on a map.

We need the item ID and asset name.  To make sure we are selecting the correct asset, we note the file type and url path.  In this project, we want to select Cloud Optimized Geotifs (COGSs) assets in the 'processed' directory.

In [4]:
print(f"Total Items at location: {len(response)}\n")

for item in response:
    print(f'Item ID: {item['id']}')
    print(item)
    for asset in item['assets']:
        print(f"    Asset Name: {asset}")
        print(f"    Asset URL: {item['assets'][asset]['href']}")
        print(f"    Asset File Type: {item['assets'][asset]['type']}")
        print("     ------------------------------------------------")
    print("\n")

Total Items at location: 1

Item ID: 20241227_140153_71_24a8-dev
{'id': '20241227_140153_71_24a8-dev', 'bbox': [-60.73502652918484, 3.8068658204904127, -60.44101581194037, 4.016034749538002], 'assets': {'20241227_140153_71_24a8_metadata_json': {'href': 'gs://planet_labs/01_jul/ae37c17a-4737-4c09-b6ed-b17157e58952/PSScene/20241227_140153_71_24a8_metadata.json', 'type': 'application/json', 'roles': ['metadata']}, '20241227_140153_71_24a8_3B_Visual_reproject_file_format_tif': {'href': 'gs://planet_labs/01_jul/ae37c17a-4737-4c09-b6ed-b17157e58952/PSScene/20241227_140153_71_24a8_3B_Visual_reproject_file_format.tif', 'type': 'image/tiff; application=geotiff; profile=cloud-optimized', 'roles': ['data', 'visual'], 'eo:bands': [{'name': 'Red', 'common_name': 'red', 'center_wavelength': 0.665, 'full_width_half_max': 0.03}, {'name': 'Green', 'common_name': 'green', 'center_wavelength': 0.565, 'full_width_half_max': 0.036}, {'name': 'Blue', 'common_name': 'blue', 'center_wavelength': 0.49, 'full_w

### View Image

Titiler has several routes that will generate an image from a STAC item.  Generally, you will need the catalog name, item id, asset name and know the number of bands.  

For example, to use the Collections/Map Viewer route, we use the inputs from the information above to create a url that will return a map viewer.

We want to create a sample map that shows the asset named 'visual' from the STAC ITEM 20250228_150538_15_2546 in the collection ares-demo

gis.roycegeo.com/pgtitiler/collections/**COLLECITON**/items/**ITEM ID**/WebMercatorQuad/map?tile_scale=1&bidx=1&bidx=2&bidx=3&assets=**ASSET NAME**&asset_bidx=**ASSET NAME**%7C1%2C2%2C3

"https://gis.roycegeo.com/pgtitiler/collections/ares-demo/items/20250228_150538_15_2546/WebMercatorQuad/map?tile_scale=1&bidx=1&bidx=2&bidx=3&assets=visual&asset_bidx=visual%7C1%2C2%2C3"

In [5]:
%%html
<iframe src="https://titiler.grvty.com/pgtitiler/collections/ares-demo/items/20250304_144759_59_24f8/WebMercatorQuad/map?tile_scale=1&bidx=1&bidx=2&bidx=3&assets=visual&asset_bidx=visual%7C1%2C2%2C3" width="1200" height="1000"></iframe>

### Register Search query

Register a PGSTAC search query and return it's ID.  You can pre-set search queries generate dynamic tilers by search id.  Here, we will create a search for items in the ares demo, in a bounding box, taken within a specified date range.

In [74]:
# search_request = {
#     # Filter collection
#     "collections": ["ares-demo"],
#     # limit bounds of the known items (note: the bbox will also be used in the tilejson response)
#     "bbox": bounds,
# 	"datetime": "2024-11-28T01:01:01Z/2024-11-30T01:01:01Z",
#     "filter-lang": "cql-json",
# }

# response = httpx.post(
#     f"{endpoint}/searches/register",
#     json=search_request,
# ).json()
# print(response)

# searchid = response["id"]
searchid = "e1f9f3d794ac9669cf68e040e118cfbc"

### Get Search Metadata

In [75]:
info_response = httpx.get(f"{endpoint}/searches/{searchid}/info").json()
print(json.dumps(info_response, indent=4))

{
    "search": {
        "hash": "e1f9f3d794ac9669cf68e040e118cfbc",
        "search": {
            "bbox": [
                -63.58651954366129,
                2.1766122457878,
                -63.25108334986687,
                2.4148986049920276
            ],
            "datetime": "2024-11-28T01:01:01Z/2024-11-30T01:01:01Z",
            "collections": [
                "ares-demo"
            ],
            "filter-lang": "cql-json"
        },
        "_where": "collection = ANY ('{ares-demo}')  AND  datetime <= '2024-11-30 01:01:01+00'::timestamptz AND end_datetime >= '2024-11-28 01:01:01+00'::timestamptz  AND st_intersects(geometry, '0103000020E61000000100000005000000BB3E891213CB4FC08DDA5DAEB3690140BB3E891213CB4FC035C61C5CB6510340C91FCC7F23A04FC035C61C5CB6510340C91FCC7F23A04FC08DDA5DAEB3690140BB3E891213CB4FC08DDA5DAEB3690140')",
        "orderby": "datetime DESC, id DESC",
        "lastused": "2025-03-17T17:53:21.541131Z",
        "usecount": 2,
        "metadata": {
       

### Get TileJSON

Note: to return a valid tilejson document you'll need to pass either the `assets` or `expression` option.  Here we know that the name of the assets we want is 'visual.'

In [76]:
tj_response = httpx.get(
   f"{endpoint}/searches/{searchid}/WebMercatorQuad/tilejson.json?assets=visual"
).json()
# tj_response = httpx.get(
#     f"https://gis.roycegeo.com/pgtitiler/collections/ares-demo/items/20250228_150538_15_2546/WGS1984Quad/tilejson.json?bidx=1&bidx=2&bidx=3&assets=visual&asset_bidx=visual%7C1%2C2%2C3"
# ).json()

print(json.dumps(tj_response, indent=4))

{
    "tilejson": "2.2.0",
    "name": "e1f9f3d794ac9669cf68e040e118cfbc",
    "version": "1.0.0",
    "scheme": "xyz",
    "tiles": [
        "https://gis.roycegeo.com/pgtitiler/searches/e1f9f3d794ac9669cf68e040e118cfbc/tiles/WebMercatorQuad/{z}/{x}/{y}?assets=visual"
    ],
    "minzoom": 0,
    "maxzoom": 24,
    "bounds": [
        -63.58651954366129,
        2.1766122457878,
        -63.25108334986687,
        2.4148986049920276
    ],
    "center": [
        -63.41880144676408,
        2.2957554253899137,
        0
    ]
}


## Load tiles

Use the TileJson create a layer on a map.

In [77]:
m = Map(
    location=((bounds[1] + bounds[3]) / 2, (bounds[0] + bounds[2]) / 2), zoom_start=14
)

geo_json = GeoJson(
    data=geojson,
    style_function=lambda x: {
        "opacity": 1,
        "dashArray": "1",
        "fillOpacity": 0,
        "weight": 1,
    },
)
geo_json.add_to(m)

aod_layer = TileLayer(
    tiles=tj_response["tiles"][0],
    attr="Mosaic",
    min_zoom=9,
    max_zoom=18,
    max_native_zoom=12,
)
aod_layer.add_to(m)
m

### Add Image Chip

Use the Collections / Items path.

gis.roycegeo.com/pgtitiler/collections/**Collection Name**/items/**Item ID**/bbox/**Bounding Box**/**HxW.File Format**/**Band Info**/assets=**Asset Name**/**Asset Band Info**

In [6]:
from IPython.display import Image
import IPython.display as Disp
image_url = "https://titiler.grvty.com/pgtitiler/collections/royce-imagery/items/20240202_113458_SN29_RR_VISUAL_MS/bbox/37.372337,55.820437,37.391499,55.829923/256x256.png?bidx=1&bidx=2&bidx=3&assets=processed_image_0&asset_bidx=processed_image_0%7C1%2C2%2C3"
Image(url=image_url) 

#<img src=image_url>


### WMTS Get Capabilities

In [7]:
response = httpx.get(
    f"{endpoint}/searches/e1f9f3d794ac9669cf68e040e118cfbc/WGS1984Quad/WMTSCapabilities.xml?tile_format=png&tile_scale=1&use_epsg=false&assets=visual"
)

# https://gis.roycegeo.com/pgtitiler/collections/royce-imagery/items/20240202_113458_SN29_RR_VISUAL_MS/WGS1984Quad/tilejson.json?tile_format=npy&tile_scale=1&bidx=1&bidx=2&bidx=3&assets=processed_image_0&asset_bidx=processed_image_0%7C1%2C2%2C3

print(response.text)

<Capabilities xmlns="http://www.opengis.net/wmts/1.0" xmlns:ows="http://www.opengis.net/ows/1.1" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:gml="http://www.opengis.net/gml" xsi:schemaLocation="http://www.opengis.net/wmts/1.0 http://schemas.opengis.net/wmts/1.0/wmtsGetCapabilities_response.xsd" version="1.0.0">
    <ows:ServiceIdentification>
        <ows:Title></ows:Title>
        <ows:ServiceType>OGC WMTS</ows:ServiceType>
        <ows:ServiceTypeVersion>1.0.0</ows:ServiceTypeVersion>
    </ows:ServiceIdentification>
    <ows:OperationsMetadata>
        <ows:Operation name="GetCapabilities">
            <ows:DCP>
                <ows:HTTP>
                    <ows:Get xlink:href="https://titiler.grvty.com/pgtitiler/searches/e1f9f3d794ac9669cf68e040e118cfbc/WGS1984Quad/WMTSCapabilities.xml?tile_format=png&amp;tile_scale=1&amp;use_epsg=false&amp;assets=visual">
                        <ows:Constraint name="GetEncoding">
       

### Get NumPY Tile  

See below:

https://github.com/planetlabs/numpytiles-spec/tree/main/1.0.0

In [80]:
import httpx
import mercantile
from io import BytesIO
import numpy

r = httpx.get(f"{endpoint}/collections/ares-demo/items/20250228_150538_15_2546/WebMercatorQuad/tilejson.json?tile_format=png&tile_scale=1&minzoom=10&maxzoom=18&bidx=1&bidx=2&bidx=3&assets=visual&asset_bidx=visual%7C1%2C2%2C3").json()
print(r)

# Get a list of tiles for minzoom + 2

tiles = list(mercantile.tiles(*r["bounds"], r["minzoom"] + 2))
print(tiles)

# Call TiTiler endpoint using the first tile

tile = tiles[0]
r = httpx.get(
    f"{endpoint}/collections/ares-demo/items/20250228_150538_15_2546/tiles/WebMercatorQuad/{tile.z}/{tile.x}/{tile.y}.npy?scale=1&bidx=1&bidx=2&bidx=3&assets=visual&asset_bidx=visual%7C1%2C2%2C3"
)

# Load result using numpy.load

arr = numpy.load(BytesIO(r.content))
print(type(arr))
print(arr.shape)

# By default we put the data and the mask in the same array

tile, mask = arr[0:-1], arr[-1]

print(tile.shape)


{'tilejson': '2.2.0', 'version': '1.0.0', 'scheme': 'xyz', 'tiles': ['https://gis.roycegeo.com/pgtitiler/collections/ares-demo/items/20250228_150538_15_2546/tiles/WebMercatorQuad/{z}/{x}/{y}@1x.png?bidx=1&bidx=2&bidx=3&assets=visual&asset_bidx=visual%7C1%2C2%2C3'], 'minzoom': 10, 'maxzoom': 18, 'bounds': [-63.57583209829002, 2.219970784751004, -63.23360293530877, 2.4647709380868346], 'center': [-63.404717516799394, 2.342370861418919, 10]}
[Tile(x=1324, y=2019, z=12), Tile(x=1324, y=2020, z=12), Tile(x=1324, y=2021, z=12), Tile(x=1324, y=2022, z=12), Tile(x=1325, y=2019, z=12), Tile(x=1325, y=2020, z=12), Tile(x=1325, y=2021, z=12), Tile(x=1325, y=2022, z=12), Tile(x=1326, y=2019, z=12), Tile(x=1326, y=2020, z=12), Tile(x=1326, y=2021, z=12), Tile(x=1326, y=2022, z=12), Tile(x=1327, y=2019, z=12), Tile(x=1327, y=2020, z=12), Tile(x=1327, y=2021, z=12), Tile(x=1327, y=2022, z=12), Tile(x=1328, y=2019, z=12), Tile(x=1328, y=2020, z=12), Tile(x=1328, y=2021, z=12), Tile(x=1328, y=2022, z=1

### External Data

In [81]:
url = "https://opendata.digitalglobe.com/events/mauritius-oil-spill/post-event/2020-08-12/105001001F1B5B00/105001001F1B5B00.tif"

# Fetch File Metadata to get min/max rescaling values (because the file is stored as float32)
r = httpx.get(
    f"{endpoint}/external/info.geojson",
    params={
        "url": url,
    },
).json()

print(json.dumps(r, indent=4))

{
    "bbox": [
        57.664053823239804,
        -20.55473177712791,
        57.84021477996238,
        -20.25261582755764
    ],
    "type": "Feature",
    "geometry": {
        "type": "Polygon",
        "coordinates": [
            [
                [
                    57.664053823239804,
                    -20.55473177712791
                ],
                [
                    57.84021477996238,
                    -20.55473177712791
                ],
                [
                    57.84021477996238,
                    -20.25261582755764
                ],
                [
                    57.664053823239804,
                    -20.25261582755764
                ],
                [
                    57.664053823239804,
                    -20.55473177712791
                ]
            ]
        ]
    },
    "properties": {
        "bounds": [
            57.664053823239804,
            -20.55473177712791,
            57.84021477996238,
            -20.2